# 데이터 준비

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.listdir('./2021_to_2025')

['NewsResult_20210101-20210106.xlsx',
 'NewsResult_20210107-20210111.xlsx',
 'NewsResult_20210112-20210115.xlsx',
 'NewsResult_20210116-20210120.xlsx',
 'NewsResult_20210121-20210125.xlsx',
 'NewsResult_20210126-20210129.xlsx',
 'NewsResult_20210130-20210203.xlsx',
 'NewsResult_20210204-20210208.xlsx',
 'NewsResult_20210209-20210215.xlsx',
 'NewsResult_20210216-20210219.xlsx',
 'NewsResult_20210220-20210224.xlsx',
 'NewsResult_20210225-20210302.xlsx',
 'NewsResult_20210303-20210308.xlsx',
 'NewsResult_20210309-20210313.xlsx',
 'NewsResult_20210314-20210318.xlsx',
 'NewsResult_20210319-20210324.xlsx',
 'NewsResult_20210325-20210329.xlsx',
 'NewsResult_20210330-20210402.xlsx',
 'NewsResult_20210403-20210407.xlsx',
 'NewsResult_20210408-20210412.xlsx',
 'NewsResult_20210413-20210416.xlsx',
 'NewsResult_20210417-20210421.xlsx',
 'NewsResult_20210422-20210427.xlsx',
 'NewsResult_20210428-20210502.xlsx',
 'NewsResult_20210503-20210507.xlsx',
 'NewsResult_20210508-20210512.xlsx',
 'NewsResult

In [3]:
df = pd.read_excel('./2021_to_2025/NewsResult_20210101-20210106.xlsx')

for path in os.listdir('./2021_to_2025')[1:]:
    new_df = pd.read_excel('./2021_to_2025/' + path)
    df = pd.concat([df, new_df], axis=0)
df.shape

C:\Users\samsung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\samsung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\samsung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\samsung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\samsung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workboo

(5325676, 19)

In [6]:
환율 = pd.read_csv('어쩌면최종데이터.csv')

# ✅ 결측값 개수 및 비율 계산
null_df = pd.DataFrame({
    '결측값 개수': 환율.isnull().sum(),
    '결측 비율 (%)': 환율.isnull().mean() * 100
})

# ✅ 'alpha'가 컬럼명에 포함된 것 중에서만 결측 비율 5% 이상인 컬럼 필터링
alpha_cols = [col for col in 환율.columns if 'alpha' in col]
alpha_null_df = null_df.loc[alpha_cols]
drop_cols = alpha_null_df[alpha_null_df['결측 비율 (%)'] >= 5].index.tolist()

# ✅ 해당 alpha 컬럼들만 삭제
환율 = 환율.drop(columns=drop_cols)

# ✅ 남은 결측값 전체 삭제 (단, alpha 외 컬럼 포함)
환율 = 환율.dropna()

환율 = 환율.sort_values(by='Date').reset_index(drop=True)
print('완료')

완료


In [4]:
print('수집한 데이터는 약', int(df.shape[0]/10000), '만 건입니다')

수집한 데이터는 약 532 만 건입니다


In [9]:
환율[['Date','미환율_종가','한국EPU']]

,Date,미환율_종가,한국EPU
0,2009-10-09,1164.30,83.677435
1,2009-10-13,1166.00,83.677435
2,2009-10-14,1157.50,83.677435
3,2009-10-15,1157.00,83.677435
4,2009-10-16,1162.10,83.677435
...,...,...,...
2679,2025-05-21,1374.83,136.720000
2680,2025-05-27,1375.80,136.720000
2681,2025-05-28,1374.59,136.720000
2682,2025-05-29,1371.39,136.720000


In [11]:
df.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

# EPU 직접 수집한 데이터로 계산
## 처리 순서
- 언론사 n개 정의
- 경제, 불확실성, 정책 카테고리에 각각 하나의 용어를 공동으로 포함하는 기사를 수집
- 기간에 맞게 뉴스 기사에 대한 데이터를 수집
- 신문사, 월별로 관련 기사 수를 집계 후, 동일 신문의 총 기사 수로 집계하여 상대 빈도 수를 얻음
- 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
- 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
- EPU 디수를 표본 기간 중 평균 100으로 정규화

# 월별, 논문에 있는 EPU만 했을 때
- 직접개발 0.361912	< barker 0.412254

In [5]:
# 경제
e = ['경제','경기']
# 정책
p = ['불확실','불확실성','리스크','불투명','불안','우려','걱정']
# 불확실
u = '정부 청와대 국무회의 국회 의회 당국 한국은행 한은 중앙은행 기획재정부 기재부 금융위원회 금융위 정책 재정 입법 법안 법률 예산 재정 세금 규제 규정 적자 부족 부채 채무 연방준비제도 연준 Fed FRB 구조개혁 구조조정'.split()
금융정책_불확실성_지수 =['통화정책','한국은행','한은','금융완화','추가완화','양적완화','질적완화','통화긴축','유동성긴축','마이너스 금리','통화할인율','재할인율','통화 운용','공개시장조작','물가안정목표','물가목표','금융통화위원회','금통위','금융통화운영위원회','금융통의','기준금리','정책금리','금리정책','통화당국','환금리']
재정정책_불확실성_지수 = [
    '재정정책', '정부 예산', '추가경정예산', '추경', '일반 회계', '특별 회계', '재정적자', '재정수지',
    '세금', '조세', '정부지출', '재정지출', '사회보장성 지출', '국민연금 보험료', '국민건강 보험료',
    '의료비 지출', '간병비 지출', '의료보수수가', '의료수가', '공무원급여', '공적개발원조', 'ODA',
    '국방비', '군비', '국채 발행 잔여', '공공부문 부채', '재정부채', '국채', '정부부채', '지방채',
    '경기부양', '경기부양책'
]
무역정책_불확실성_지수 = [
    '통상문제', '무역문제', '비관세장벽', '수입제한', '포괄통상법', '종합무역법',
    '무역정책', '통상정책', '무역협상', '세계무역기구', 'WTO',
    '관세 및 무역에 관한 일반협정', 'GATT', '가트', '관세 인하',
    '무역자유화', '수입자유화', '시장접근', '무역협정', '통상협정',
    '환태평양경제동반자협정', 'TPP', '경제동반자협정', '경제파트너협정',
    '경제 파트너십 협정', 'EPA', '자유무역협정', 'FTA', '무역분쟁',
    '관세', '우루과이라운드', '도하라운드', '덤핑'
]
환율정책_불확실성_지수 = [
    '외환정책', '환율정책', '시장개입', '외환시장 개입', '외환당국 개입',
    '미세조정', '환율 안정책', '환율 안정 조치', '환율 변동성 완화',
    '환율 조작', '외환보유고', '외환보유액', '달러매도', '원화 매입 개입'
]

In [6]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [7]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(165179, 19) 데이터를 사용하게 되었습니다
전체의 3.1015593137847666 %입니다.


In [8]:
df_epu.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [9]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_24592\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_24592\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [10]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [11]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [12]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [13]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [14]:
korea_epu

일자
2021-01-01     91.503674
2021-02-01     86.482249
2021-03-01     79.230107
2021-04-01     82.338196
2021-05-01     89.207983
2021-06-01     84.295773
2021-07-01     98.363333
2021-08-01     99.713534
2021-09-01     91.486980
2021-10-01     97.773026
2021-11-01     91.924101
2021-12-01     96.505819
2022-01-01    101.189722
2022-02-01    111.435023
2022-03-01    107.550723
2022-04-01     99.666269
2022-05-01    109.396993
2022-06-01    116.786035
2022-07-01    120.899059
2022-08-01    107.670848
2022-09-01    124.634836
2022-10-01    122.080064
2022-11-01     97.189688
2022-12-01    107.822255
2023-01-01    113.122614
2023-02-01     88.431665
2023-03-01    107.910167
2023-04-01     98.519289
2023-05-01     89.501759
2023-06-01     86.944684
2023-07-01     83.704666
2023-08-01    110.062530
2023-09-01     89.081849
2023-10-01     99.834353
2023-11-01     86.039305
2023-12-01     92.860226
2024-01-01    103.386924
2024-02-01     90.507057
2024-03-01     84.719063
2024-04-01    101.0824

In [15]:
pd.DataFrame(korea_epu).reset_index().to_csv('one_2021_to_2025.csv',index=False)

# 논문에 있던 다른 키워드 추가
- 직접개발 0.370857	< barker 0.412254

In [4]:
# 1) 경제(E): 거시경기·실물경제를 직접 지칭
e = [
    "경제", "경기"
]

# 2) 정책(P): 정부·의회·당국 및 모든 정책·제도·규제·조치
p = [
    # ─ 기본 정책 키워드 ─
    "정부", "청와대", "국무회의", "국회", "의회", "당국",
    "한국은행", "한은", "중앙은행",
    "기획재정부", "기재부", "금융위원회", "금융위",
    "정책", "재정", "입법", "법안", "법률", "예산",
    "세금", "규제", "규정", "적자", "부채", "채무",
    "연방준비제도", "연준", "Fed", "FRB",
    "구조개혁", "구조조정",

    # ─ 통화·금융 정책 ─
    "통화정책", "금융완화", "추가완화", "양적완화", "질적완화",
    "통화긴축", "유동성긴축", "마이너스 금리", "통화할인율", "재할인율",
    "통화 운용", "공개시장조작", "물가안정목표", "물가목표",
    "금융통화위원회", "금통위", "금융통화운영위원회", "금융통의",
    "기준금리", "정책금리", "금리정책", "통화당국", "환금리",

    # ─ 재정 정책 ─
    "재정정책", "정부 예산", "추가경정예산", "추경", "일반 회계", "특별 회계",
    "재정적자", "재정수지", "정부지출", "재정지출",
    "사회보장성 지출", "국민연금 보험료", "국민건강 보험료",
    "의료비 지출", "간병비 지출", "의료보수수가", "의료수가",
    "공무원급여", "공적개발원조", "ODA",
    "국방비", "군비",
    "국채 발행 잔여", "공공부문 부채", "재정부채", "국채", "정부부채", "지방채",
    "경기부양", "경기부양책",

    # ─ 무역·통상 정책 ─
    "통상문제", "무역문제", "비관세장벽", "수입제한",
    "포괄통상법", "종합무역법",
    "무역정책", "통상정책", "무역협상",
    "세계무역기구", "WTO",
    "관세 및 무역에 관한 일반협정", "GATT", "가트",
    "관세 인하", "무역자유화", "수입자유화", "시장접근",
    "무역협정", "통상협정",
    "환태평양경제동반자협정", "TPP",
    "경제동반자협정", "경제파트너협정", "경제 파트너십 협정", "EPA",
    "자유무역협정", "FTA",
    "무역분쟁", "관세", "우루과이라운드", "도하라운드", "덤핑",

    # ─ 환율 정책 ─
    "외환정책", "환율정책", "시장개입", "외환시장 개입", "외환당국 개입",
    "미세조정", "환율 안정책", "환율 안정 조치", "환율 변동성 완화",
    "환율 조작", "외환보유고", "외환보유액", "달러매도", "원화 매입 개입"
]

# 3) 불확실(U): 모호성·위험·걱정·우려를 표현
u = [
    "불확실", "불확실성", "리스크", "불투명",
    "불안", "우려", "걱정"
]


In [5]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [6]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(161725, 19) 데이터를 사용하게 되었습니다
전체의 3.0367036973334467 %입니다.


In [7]:
df_epu.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [8]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_15880\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_15880\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [9]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [10]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [11]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [12]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [13]:
korea_epu

일자
2021-01-01     91.138083
2021-02-01     85.831162
2021-03-01     79.310385
2021-04-01     81.660729
2021-05-01     88.909046
2021-06-01     83.240482
2021-07-01     97.452929
2021-08-01     98.474541
2021-09-01     90.869488
2021-10-01     96.828495
2021-11-01     90.827044
2021-12-01     96.189968
2022-01-01    101.305081
2022-02-01    109.842051
2022-03-01    107.137651
2022-04-01    100.499773
2022-05-01    110.389186
2022-06-01    116.940802
2022-07-01    120.938735
2022-08-01    107.926226
2022-09-01    124.752068
2022-10-01    122.888946
2022-11-01     95.460556
2022-12-01    107.761614
2023-01-01    112.334067
2023-02-01     88.333533
2023-03-01    108.461061
2023-04-01     98.888503
2023-05-01     89.475868
2023-06-01     86.636233
2023-07-01     83.454140
2023-08-01    109.839989
2023-09-01     88.008226
2023-10-01    100.808110
2023-11-01     86.218273
2023-12-01     91.961583
2024-01-01    103.206521
2024-02-01     89.037432
2024-03-01     83.926412
2024-04-01    101.4699

In [14]:
pd.DataFrame(korea_epu).reset_index().to_csv('two_2021_to_2025.csv',index=False)

# KDI 한국개발연구원 키워드만
- 직접개발 0.500216 > barker 0.412254 	

In [15]:
df.to_csv('df_2021_to_2025.csv')

In [16]:
# 1) 경제(E): 거시경기실물경제를 직접 지칭
e = [
    '경제','경제의','상업','무역'
]

# 2) 정책(P): 정부·의회·당국 및 모든 정책·제도·규제·조치
p = [
    '정부', '청와대', '국회', '당국', '한국은행', '중앙은행', '한은', '기재부', '기획재정부', '세계무역기구'
    , 'WTO', '제정', '재정법', '입법', '세금', '규제', '규정', '적자', '부족'
]

# 3) 불확실(U): 모호성·위험·걱정·우려를 표현
u = [
    '불확실성','불확'
]


In [17]:
import re

cols = ['키워드', '특성추출(가중치순 상위 50개)', '본문', '제목']          # 검색 대상 열
joined = df[cols].fillna('').agg(' '.join, axis=1)                          # 네 열을 하나로 합침

cond1 = joined.str.contains('|'.join(map(re.escape, e)))                    # 경제·경기
cond2 = joined.str.contains('|'.join(map(re.escape, p)))                    # 불확실 관련
cond3 = joined.str.contains('|'.join(map(re.escape, u)))                    # 정책(정부 등)
                 # 정책(정부 등)

# 세 조건을 모두 만족하는 기사만 추리기
df_epu = df[cond1&cond2&cond3]

In [18]:
print(df_epu.shape, '데이터를 사용하게 되었습니다')
print('전체의', df_epu.shape[0]/df.shape[0] *100, '%입니다.')

(8068, 19) 데이터를 사용하게 되었습니다
전체의 0.1514925053645772 %입니다.


In [19]:
df_epu.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [20]:
# 신문사 월별로 집계
df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')
집계 = pd.DataFrame(df_epu.groupby(['일자','언론사']).size().reset_index())
집계.columns = ['일자','언론사','epu기사수']

C:\Users\samsung\AppData\Local\Temp\ipykernel_15880\353623932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자'] = pd.to_datetime(df_epu['일자'], format='%Y%m%d')
C:\Users\samsung\AppData\Local\Temp\ipykernel_15880\353623932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epu['일자']= df_epu['일자'].dt.strftime('%Y-%m-01')


In [21]:
# 각 신문의 상대 기사 수
전체집계 = pd.DataFrame(df.groupby(['일자','언론사']).size().reset_index())
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y%m%d')
전체집계['일자'] = 전체집계['일자'].dt.strftime('%Y-%m-01')
전체집계.columns =  ['일자','언론사','전체기사수']
전체집계 = 전체집계.groupby(['일자','언론사'])['전체기사수'].sum().reset_index()
전체집계= 전체집계.merge(집계, on=['일자','언론사'])
전체집계['상대빈도수'] = 전체집계['epu기사수']/전체집계['전체기사수']

In [22]:
# 각 신문의 상대 EPU 수를 전체 표본 기간의 단위 표준 편차로 표준화
전체집계['표준화빈도'] = 전체집계['상대빈도수'] / 전체집계.groupby('언론사')['상대빈도수'].transform('std')

In [23]:
# 신문의 표준화 시리즈를 월별로 평균하여 전체 월간 한국 EPU지수
전체집계['일자'] = pd.to_datetime(전체집계['일자'], format='%Y-%m-01')

# ② 신문 간 단순 평균 → 월별 EPU
monthly_epu = (전체집계
               .groupby('일자')['표준화빈도']
               .mean())

In [24]:
# EPU 지수를 표본 기간 중 평균 100으로 정규화
korea_epu = (monthly_epu / monthly_epu.mean() * 100).rename('Korea_EPU')

In [25]:
korea_epu

일자
2021-01-01     55.525073
2021-02-01     49.618401
2021-03-01     37.079625
2021-04-01     52.526443
2021-05-01     42.053629
2021-06-01     37.726860
2021-07-01     47.043190
2021-08-01     53.351913
2021-09-01     37.771488
2021-10-01     58.762105
2021-11-01     72.949145
2021-12-01     79.792822
2022-01-01     57.668884
2022-02-01     82.382060
2022-03-01    114.333021
2022-04-01     67.560699
2022-05-01     77.554822
2022-06-01     67.948607
2022-07-01     75.245002
2022-08-01     53.340753
2022-09-01     74.641107
2022-10-01     79.786455
2022-11-01     54.461177
2022-12-01     86.624920
2023-01-01     78.874009
2023-02-01     77.962944
2023-03-01     84.648890
2023-04-01     91.417825
2023-05-01     51.976116
2023-06-01     45.563229
2023-07-01     43.132802
2023-08-01     62.408492
2023-09-01     59.235797
2023-10-01    126.546007
2023-11-01     78.856925
2023-12-01     74.697770
2024-01-01     64.349981
2024-02-01     40.132746
2024-03-01     36.021932
2024-04-01     89.4703

In [26]:
pd.DataFrame(korea_epu).reset_index().to_csv('three_2021_to_2025.csv',index=False)